# Customer Segmentation - RFM Analysis

## Import Dependencies

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')                              ## Matplotlib style
import warnings
warnings.filterwarnings('ignore')                            ## Silence the warnings

## Load the data and process

In [ ]:
df=pd.read_csv(r"/home/jess/Documents/Github/Customer_Segmentation_Analysis/Sales_.csv")    ## load the file

In [ ]:
df.drop(columns=['Unnamed: 0'],inplace=True)                                     ## Drop the old index column
df.Order_date = df.Order_date.astype('datetime64')                               ## Convert the date column from string to datetime format

In [ ]:
df = df[['Customer_ID','Order_ID','Order_date','Quantity','Price']]                  ## Order the columns
df['Revenue'] = df.Quantity * df.Price                                             ## Create a new revenue column

In [ ]:
RFM =df.groupby('Customer_ID').agg({'Order_ID':'count','Revenue':'sum','Order_date':lambda x :(df.Order_date.max()-x.max()).days})    ## Group the data by Customer ID and assing different aggregation functions
                                                                                                                                    ## for each column
RFM.rename(columns={'Order_ID':'Frequency','Revenue':'Monetary','Order_date':'Recency'},inplace=True)                            ## Rename the columns

In [ ]:
def condition_r(x,col='Frequency'):             ## Create a condition function to convert values in 'F' and 'M' columns
    if x < RFM[col].quantile(0.33):              ## The lower the recency value is the better for the business.
        return 3
    elif x < RFM[col].quantile(0.67):
        return 2
    else:
        return 1

In [ ]:
def condition_f_m(x,col='Frequency'):    ## Create a condition function to convert values in 'F' and 'M' columns
    if x < RFM[col].quantile(0.33):       ## The higher the monetary and frequency is the better for the business.
        return 1
    elif x < RFM[col].quantile(0.67):
        return 2
    else:
        return 3

In [ ]:
RFM['F_label']=RFM.Frequency.apply(lambda x : condition_f_m(x,col='Frequency'))   ## Apply the conversion functions to the columns
RFM['M_label']=RFM.Monetary.apply(lambda x : condition_f_m(x,col='Monetary'))
RFM['R_label']=RFM.Recency.apply(lambda x : condition_r(x,col='Recency'))

In [ ]:
RFM.head(3)

Newly created labels show the ranking for each row. 4 is the best number. The best row is the one with three '4' values. Such row represents a customer who makes purchase frequently, who has spent high amount of money and who purchased very recently.

In [ ]:
RFM['Score'] = RFM[['F_label','M_label','R_label']].sum(axis=1)                      ## Add the values from the three columns
RFM = RFM.sort_values('Score',ascending=False)                           ## Sort  the dataframe by the newly created column

In [ ]:
RFM['Segment'] = pd.qcut(RFM.Score,3,['Diamond','Gold','Silver'][::-1])    ## Divide the Score values into 4 different parts
RFM.head()

In [ ]:
RFM.Segment.value_counts()                     ## The number of customers belonging to each segment

## RFM Exploratory Data Analysis

In [ ]:
fig,axes = plt.subplots(1,4, figsize=(18,4))                            ## Plotting the distribution of customers by each feature
plt.suptitle('Distribution of Features')                            ## title
sns.histplot(RFM.Frequency,ax = axes[0],color='brown',kde=True)
sns.histplot(RFM.Recency,ax = axes[1],color='green',kde=True)
sns.histplot(RFM.Monetary,ax = axes[2],kde=True)
sns.histplot(RFM.Segment,ax = axes[3],color='y')
plt.show()

## Correlation Between The Features

In [ ]:
cor = RFM.corr()                                           ## correlation matrix
sns.heatmap(cor,annot=True, cmap= 'viridis',annot_kws={'size':12})
plt.xticks(rotation = 45)
plt.show()

In [ ]:
fig,axes = plt.subplots(1,3,figsize=(21,4))                   ## Comparison of customers on different pair of shoes
plt.suptitle('Comparison of Features')
sns.scatterplot(x=RFM.Frequency,y=RFM.Recency,hue=RFM.Segment,ax=axes[0],legend=False)
sns.scatterplot(x=RFM.Monetary,y=RFM.Recency,hue=RFM.Segment,ax=axes[1])
sns.scatterplot(x=RFM.Monetary,y=RFM.Frequency,hue=RFM.Segment,ax=axes[2],legend=False)
plt.show()

Here, these scatterplots show the correlation between the columns;
 **Frequency** is positively correlated with **Monetary**, negatively correlated with **Recency**,
 **Recency** is negatively correlated with **Monetary** and **Frequency**,
 **Monetary** is positively correlated with **Frequency**, negatively correlated with **Recency**.

## Business Strategy According to the Findings

As a result of the analysis, there are 3 main customer segments; Diamond,Gold,Silver. General profile of a customer is as following:

**Diamond** -  This segment is the most valuable among all. Losing these customers would severely hit the profitability of the business. The behaviours of these customers show that they are happy with the quality, price and other conditions. Therefore, customers from this segment should be provided with more than ordinary promotions to boost the relationship even more. An example could be giving early access for new productions, involving them into the decision-making processes of the business.

**Gold** - This segment is also happy with the business operations, however there are still some subtle improvements to make. Some may show relatively low frequency, some may have a low purchase volume. It is crucial to detect the improvable part and create solutions. For example; If the frequency is low, making analysis of substitute products on the market would be a good idea, because low frequency generally is an indicator of that the customer buys the products of other brands from time to time.

**Silver** - Silver customers are the hardest segment to regain.Renewed price lists, promotions, different accessibility options could be tried for strengthen the relationship. Example; if the problem is high recency, it is an amazing way to remind the products of the business by reaching out to the customers with new deals and promotions so that they get back to the old routine. Retention costs are high for this segment. They require more effort, more money and more time to be retained. It is not always profitable to spend resources for this segment. Only if the business aims to get more market share and profitability is not the priority, this segment should be targeted. Low cost activities should be chosen to detect what caused them to avoid the products.